# FINRA API Authentication Debug

This notebook tests FINRA API authentication to diagnose the 400 Bad Request error.

In [ ]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from root .env
load_dotenv('.env')

FINRA_API_KEY = os.getenv('FINRA_API_KEY')
FINRA_API_SECRET = os.getenv('FINRA_API_SECRET')

print(f"API Key loaded: {FINRA_API_KEY[:10]}..." if FINRA_API_KEY else "API Key NOT FOUND")
print(f"API Secret loaded: {FINRA_API_SECRET[:5]}..." if FINRA_API_SECRET else "API Secret NOT FOUND")

## Test 1: Token URL with grant_type in body only

FINRA OAuth2 typically expects `grant_type` in the request body, not the URL.

In [ ]:
# Token URL WITHOUT grant_type in query string
TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

response = requests.post(
    TOKEN_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Headers: {dict(response.headers)}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

## Test 2: Token URL with grant_type in query string

Some APIs expect it in the URL instead.

In [ ]:
# Token URL WITH grant_type in query string
TOKEN_URL_WITH_GRANT = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token?grant_type=client_credentials"

response = requests.post(
    TOKEN_URL_WITH_GRANT,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

## Test 3: Using headers instead of Basic Auth

Some FINRA endpoints use custom headers for authentication.

In [ ]:
TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

# Try with X-API-KEY headers instead of Basic Auth
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "X-API-KEY": FINRA_API_KEY,
    "X-API-SECRET": FINRA_API_SECRET
}

response = requests.post(
    TOKEN_URL,
    headers=headers,
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

## Test 4: Direct API call without token (API Key auth)

Some FINRA APIs allow direct API key authentication without OAuth token.

In [ ]:
# Try direct API call with API key headers
OTC_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"

headers = {
    "Accept": "application/json",
    "X-API-KEY": FINRA_API_KEY
}

response = requests.get(
    OTC_URL,
    headers=headers,
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

## Test 5: Check FINRA Gateway API directly

Try the FINRA Gateway API endpoint format.

In [ ]:
# FINRA Gateway API format
GATEWAY_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"

# Using Basic Auth with the API credentials
response = requests.post(
    GATEWAY_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    headers={"Accept": "application/json", "Content-Type": "application/json"},
    json={},  # Empty body for now
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:1000] if response.text else 'No response body'}")

## Test 6: Correct FINRA CAT/OTC API format

Based on FINRA documentation, try the proper request format.

In [ ]:
import base64

# Encode credentials for Authorization header
credentials = f"{FINRA_API_KEY}:{FINRA_API_SECRET}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(
    TOKEN_URL,
    headers=headers,
    data="grant_type=client_credentials",
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

if response.status_code == 200:
    token_data = response.json()
    print(f"\nAccess Token: {token_data.get('access_token', 'N/A')[:50]}...")

## Summary

Based on the tests above, identify which authentication method works and update the code accordingly.